In [2]:
from datasets import load_dataset

data = load_dataset("microsoft/FStarDataSet")
train_data = data["train"]
eval_data = data["validation"]
test_data = data["test"]

intra_project_test = test_data.filter(lambda x: x["isa_cross_project_example"] == False)
cross_project_test = test_data.filter(lambda x: x["isa_cross_project_example"] == True)


c:\Users\sfakhoury\AppData\Local\miniconda3\envs\autogen\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Filter: 100%|██████████| 7734/7734 [00:03<00:00, 2446.93 examples/s]


In [9]:
#list columns
cross_project_test.column_names

['effect',
 'original_source_type',
 'opens_and_abbrevs',
 'isa_cross_project_example',
 'source_definition',
 'partial_definition',
 'is_div',
 'is_type',
 'is_proof',
 'completed_definiton',
 'dependencies',
 'effect_flags',
 'ideal_premises',
 'mutual_with',
 'file_context',
 'interleaved',
 'is_simply_typed',
 'file_name',
 'vconfig',
 'is_simple_lemma',
 'source_type',
 'proof_features',
 'name',
 'source',
 'verbose_type',
 'source_range']

In [27]:
print(cross_project_test["source_definition"][4])

let rec big_endian (b:bytes) : Tot (n:nat) (decreases (Seq.length b)) = 
  if Seq.length b = 0 then 0 
  else
    UInt8.v (last b) + pow2 8 * big_endian (Seq.slice b 0 (Seq.length b - 1))
